## Tornado Magnitude Prediction

Given *data about tornadoes*, let's try to predict the **magnitude** of a given tornado.

We will use three linear regression models to make our predictions.

Data source: https://www.kaggle.com/datasets/jtennis/spctornado/data

### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [2]:
data = pd.read_csv('Tornadoes_SPC_1950to2015.csv')
data

,om,yr,mo,dy,date,time,tz,st,stf,stn,mag,inj,fat,loss,closs,slat,slon,elat,elon,len,wid,fc
0,1,1950,1,3,1/3/1950,11:00:00,3,MO,29,1,3,3,0,6.00,0.0,38.77,-90.22,38.83,-90.03,9.50,150,0
1,2,1950,1,3,1/3/1950,11:55:00,3,IL,17,2,3,3,0,5.00,0.0,39.10,-89.30,39.12,-89.23,3.60,130,0
2,3,1950,1,3,1/3/1950,16:00:00,3,OH,39,1,1,1,0,4.00,0.0,40.88,-84.58,0.00,0.00,0.10,10,0
3,4,1950,1,13,1/13/1950,5:25:00,3,AR,5,1,3,1,1,3.00,0.0,34.40,-94.37,0.00,0.00,0.60,17,0
4,5,1950,1,25,1/25/1950,19:30:00,3,MO,29,2,2,5,0,5.00,0.0,37.60,-90.68,37.63,-90.65,2.30,300,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60109,607506,2015,12,28,12/28/2015,3:20:00,3,LA,22,0,1,0,0,0.00,0.0,30.08,-90.54,30.08,-90.53,0.60,150,0
60110,613277,2015,12,28,12/28/2015,4:46:00,3,AR,5,0,2,0,0,0.05,0.0,34.70,-90.92,34.78,-90.90,5.75,40,0
60111,607307,2015,12,28,12/28/2015,5:43:00,3,MS,28,0,1,0,0,0.10,0.0,31.54,-89.53,31.61,-89.49,5.59,100,0
60112,612738,2015,12,28,12/28/2015,8:30:00,3,FL,12,0,1,0,0,0.01,0.0,30.76,-87.24,30.77,-87.23,0.78,75,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60114 entries, 0 to 60113
Data columns (total 22 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   om      60114 non-null  int64  
 1   yr      60114 non-null  int64  
 2   mo      60114 non-null  int64  
 3   dy      60114 non-null  int64  
 4   date    60114 non-null  object 
 5   time    60114 non-null  object 
 6   tz      60114 non-null  int64  
 7   st      60114 non-null  object 
 8   stf     60114 non-null  int64  
 9   stn     60114 non-null  int64  
 10  mag     60114 non-null  int64  
 11  inj     60114 non-null  int64  
 12  fat     60114 non-null  int64  
 13  loss    60114 non-null  float64
 14  closs   60114 non-null  float64
 15  slat    60114 non-null  float64
 16  slon    60114 non-null  float64
 17  elat    60114 non-null  float64
 18  elon    60114 non-null  float64
 19  len     60114 non-null  float64
 20  wid     60114 non-null  int64  
 21  fc      60114 non-null  int64  
dty

### Preprocessing

In [4]:
df = data.copy()

In [5]:
# Drop date column (redundant)
df = df.drop('date', axis=1)
df

,om,yr,mo,dy,time,tz,st,stf,stn,mag,inj,fat,loss,closs,slat,slon,elat,elon,len,wid,fc
0,1,1950,1,3,11:00:00,3,MO,29,1,3,3,0,6.00,0.0,38.77,-90.22,38.83,-90.03,9.50,150,0
1,2,1950,1,3,11:55:00,3,IL,17,2,3,3,0,5.00,0.0,39.10,-89.30,39.12,-89.23,3.60,130,0
2,3,1950,1,3,16:00:00,3,OH,39,1,1,1,0,4.00,0.0,40.88,-84.58,0.00,0.00,0.10,10,0
3,4,1950,1,13,5:25:00,3,AR,5,1,3,1,1,3.00,0.0,34.40,-94.37,0.00,0.00,0.60,17,0
4,5,1950,1,25,19:30:00,3,MO,29,2,2,5,0,5.00,0.0,37.60,-90.68,37.63,-90.65,2.30,300,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60109,607506,2015,12,28,3:20:00,3,LA,22,0,1,0,0,0.00,0.0,30.08,-90.54,30.08,-90.53,0.60,150,0
60110,613277,2015,12,28,4:46:00,3,AR,5,0,2,0,0,0.05,0.0,34.70,-90.92,34.78,-90.90,5.75,40,0
60111,607307,2015,12,28,5:43:00,3,MS,28,0,1,0,0,0.10,0.0,31.54,-89.53,31.61,-89.49,5.59,100,0
60112,612738,2015,12,28,8:30:00,3,FL,12,0,1,0,0,0.01,0.0,30.76,-87.24,30.77,-87.23,0.78,75,0


In [6]:
# Drop id columns
df = df.drop(['om', 'stn'], axis=1)
df

,yr,mo,dy,time,tz,st,stf,mag,inj,fat,loss,closs,slat,slon,elat,elon,len,wid,fc
0,1950,1,3,11:00:00,3,MO,29,3,3,0,6.00,0.0,38.77,-90.22,38.83,-90.03,9.50,150,0
1,1950,1,3,11:55:00,3,IL,17,3,3,0,5.00,0.0,39.10,-89.30,39.12,-89.23,3.60,130,0
2,1950,1,3,16:00:00,3,OH,39,1,1,0,4.00,0.0,40.88,-84.58,0.00,0.00,0.10,10,0
3,1950,1,13,5:25:00,3,AR,5,3,1,1,3.00,0.0,34.40,-94.37,0.00,0.00,0.60,17,0
4,1950,1,25,19:30:00,3,MO,29,2,5,0,5.00,0.0,37.60,-90.68,37.63,-90.65,2.30,300,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60109,2015,12,28,3:20:00,3,LA,22,1,0,0,0.00,0.0,30.08,-90.54,30.08,-90.53,0.60,150,0
60110,2015,12,28,4:46:00,3,AR,5,2,0,0,0.05,0.0,34.70,-90.92,34.78,-90.90,5.75,40,0
60111,2015,12,28,5:43:00,3,MS,28,1,0,0,0.10,0.0,31.54,-89.53,31.61,-89.49,5.59,100,0
60112,2015,12,28,8:30:00,3,FL,12,1,0,0,0.01,0.0,30.76,-87.24,30.77,-87.23,0.78,75,0


In [7]:
# Drop fc column (irrelevant)
df = df.drop('fc', axis=1)
df

,yr,mo,dy,time,tz,st,stf,mag,inj,fat,loss,closs,slat,slon,elat,elon,len,wid
0,1950,1,3,11:00:00,3,MO,29,3,3,0,6.00,0.0,38.77,-90.22,38.83,-90.03,9.50,150
1,1950,1,3,11:55:00,3,IL,17,3,3,0,5.00,0.0,39.10,-89.30,39.12,-89.23,3.60,130
2,1950,1,3,16:00:00,3,OH,39,1,1,0,4.00,0.0,40.88,-84.58,0.00,0.00,0.10,10
3,1950,1,13,5:25:00,3,AR,5,3,1,1,3.00,0.0,34.40,-94.37,0.00,0.00,0.60,17
4,1950,1,25,19:30:00,3,MO,29,2,5,0,5.00,0.0,37.60,-90.68,37.63,-90.65,2.30,300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60109,2015,12,28,3:20:00,3,LA,22,1,0,0,0.00,0.0,30.08,-90.54,30.08,-90.53,0.60,150
60110,2015,12,28,4:46:00,3,AR,5,2,0,0,0.05,0.0,34.70,-90.92,34.78,-90.90,5.75,40
60111,2015,12,28,5:43:00,3,MS,28,1,0,0,0.10,0.0,31.54,-89.53,31.61,-89.49,5.59,100
60112,2015,12,28,8:30:00,3,FL,12,1,0,0,0.01,0.0,30.76,-87.24,30.77,-87.23,0.78,75


In [8]:
# Extract time features
df['time'] = pd.to_datetime(df['time'])
df['hour'] = df['time'].apply(lambda x: x.hour)
df['minute'] = df['time'].apply(lambda x: x.minute)
df = df.drop('time', axis=1)
df

/tmp/ipykernel_20669/579595714.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['time'] = pd.to_datetime(df['time'])


,yr,mo,dy,tz,st,stf,mag,inj,fat,loss,closs,slat,slon,elat,elon,len,wid,hour,minute
0,1950,1,3,3,MO,29,3,3,0,6.00,0.0,38.77,-90.22,38.83,-90.03,9.50,150,11,0
1,1950,1,3,3,IL,17,3,3,0,5.00,0.0,39.10,-89.30,39.12,-89.23,3.60,130,11,55
2,1950,1,3,3,OH,39,1,1,0,4.00,0.0,40.88,-84.58,0.00,0.00,0.10,10,16,0
3,1950,1,13,3,AR,5,3,1,1,3.00,0.0,34.40,-94.37,0.00,0.00,0.60,17,5,25
4,1950,1,25,3,MO,29,2,5,0,5.00,0.0,37.60,-90.68,37.63,-90.65,2.30,300,19,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60109,2015,12,28,3,LA,22,1,0,0,0.00,0.0,30.08,-90.54,30.08,-90.53,0.60,150,3,20
60110,2015,12,28,3,AR,5,2,0,0,0.05,0.0,34.70,-90.92,34.78,-90.90,5.75,40,4,46
60111,2015,12,28,3,MS,28,1,0,0,0.10,0.0,31.54,-89.53,31.61,-89.49,5.59,100,5,43
60112,2015,12,28,3,FL,12,1,0,0,0.01,0.0,30.76,-87.24,30.77,-87.23,0.78,75,8,30


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60114 entries, 0 to 60113
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   yr      60114 non-null  int64  
 1   mo      60114 non-null  int64  
 2   dy      60114 non-null  int64  
 3   tz      60114 non-null  int64  
 4   st      60114 non-null  object 
 5   stf     60114 non-null  int64  
 6   mag     60114 non-null  int64  
 7   inj     60114 non-null  int64  
 8   fat     60114 non-null  int64  
 9   loss    60114 non-null  float64
 10  closs   60114 non-null  float64
 11  slat    60114 non-null  float64
 12  slon    60114 non-null  float64
 13  elat    60114 non-null  float64
 14  elon    60114 non-null  float64
 15  len     60114 non-null  float64
 16  wid     60114 non-null  int64  
 17  hour    60114 non-null  int64  
 18  minute  60114 non-null  int64  
dtypes: float64(7), int64(11), object(1)
memory usage: 8.7+ MB


In [10]:
def onehot_encode(df, column):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [11]:
# One-hot encode the nominal features
for column in ['tz', 'st', 'stf']:
    df = onehot_encode(df, column=column)

In [12]:
df

,yr,mo,dy,mag,inj,fat,loss,closs,slat,slon,elat,elon,len,wid,hour,minute,tz_0,tz_3,tz_6,tz_9,st_AK,st_AL,st_AR,st_AZ,st_CA,st_CO,st_CT,st_DC,st_DE,st_FL,st_GA,st_HI,st_IA,st_ID,st_IL,st_IN,st_KS,st_KY,st_LA,st_MA,st_MD,st_ME,st_MI,st_MN,st_MO,st_MS,st_MT,st_NC,st_ND,st_NE,st_NH,st_NJ,st_NM,st_NV,st_NY,st_OH,st_OK,st_OR,st_PA,st_PR,st_RI,st_SC,st_SD,st_TN,st_TX,st_UT,st_VA,st_VT,st_WA,st_WI,st_WV,st_WY,stf_1,stf_2,stf_4,stf_5,stf_6,stf_8,stf_9,stf_10,stf_11,stf_12,stf_13,stf_15,stf_16,stf_17,stf_18,stf_19,stf_20,stf_21,stf_22,stf_23,stf_24,stf_25,stf_26,stf_27,stf_28,stf_29,stf_30,stf_31,stf_32,stf_33,stf_34,stf_35,stf_36,stf_37,stf_38,stf_39,stf_40,stf_41,stf_42,stf_44,stf_45,stf_46,stf_47,stf_48,stf_49,stf_50,stf_51,stf_53,stf_54,stf_55,stf_56,stf_72
0,1950,1,3,3,3,0,6.00,0.0,38.77,-90.22,38.83,-90.03,9.50,150,11,0,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,1950,1,3,3,3,0,5.00,0.0,39.10,-89.30,39.12,-89.23,3.60,130,11,55,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,1950,1,3,1,1,0,4.00,0.0,40.88,-84.58,0.00,0.00,0.10,10,16,0,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,1950,1,13,3,1,1,3.00,0.0,34.40,-94.37,0.00,0.00,0.60,17,5,25,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,1950,1,25,2,5,0,5.00,0.0,37.60,-90.68,37.63,-90.65,2.30,300,19,30,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fals

In [13]:
# Split df into X and y
y = df['mag']
X = df.drop('mag', axis=1)

In [14]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=89)

In [15]:
X_train.shape, X_test.shape

((42079, 123), (18035, 123))

In [16]:
# Scale X
scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)

In [17]:
X_train

,yr,mo,dy,inj,fat,loss,closs,slat,slon,elat,elon,len,wid,hour,minute,tz_0,tz_3,tz_6,tz_9,st_AK,st_AL,st_AR,st_AZ,st_CA,st_CO,st_CT,st_DC,st_DE,st_FL,st_GA,st_HI,st_IA,st_ID,st_IL,st_IN,st_KS,st_KY,st_LA,st_MA,st_MD,st_ME,st_MI,st_MN,st_MO,st_MS,st_MT,st_NC,st_ND,st_NE,st_NH,st_NJ,st_NM,st_NV,st_NY,st_OH,st_OK,st_OR,st_PA,st_PR,st_RI,st_SC,st_SD,st_TN,st_TX,st_UT,st_VA,st_VT,st_WA,st_WI,st_WV,st_WY,stf_1,stf_2,stf_4,stf_5,stf_6,stf_8,stf_9,stf_10,stf_11,stf_12,stf_13,stf_15,stf_16,stf_17,stf_18,stf_19,stf_20,stf_21,stf_22,stf_23,stf_24,stf_25,stf_26,stf_27,stf_28,stf_29,stf_30,stf_31,stf_32,stf_33,stf_34,stf_35,stf_36,stf_37,stf_38,stf_39,stf_40,stf_41,stf_42,stf_44,stf_45,stf_46,stf_47,stf_48,stf_49,stf_50,stf_51,stf_53,stf_54,stf_55,stf_56,stf_72
29420,0.116889,1.265042,1.388857,-0.085786,-0.060007,-0.109951,-0.027623,-0.580869,-0.969422,-1.106666,1.117762,-0.281374,-0.145029,-0.438211,-1.147695,-0.012899,0.023386,-0.019503,0.0,-0.006894,-0.182547,-0.171773,-0.062743,-0.084596,-0.189656,-0.039935,0.0,-0.030458,-0.237009,-0.160442,-0.026711,-0.205311,-0.057361,-0.201510,-0.154418,-0.265918,-0.124669,-0.181153,-0.05303,-0.075897,-0.044456,-0.129686,-0.172505,-0.192196,-0.186063,-0.083449,-0.145895,-0.157059,-0.218893,-0.039935,-0.048068,-0.095967,-0.040527,-0.083304,-0.126233,-0.255152,-0.041687,-0.111967,-0.018884,-0.012899,-0.127201,-0.17367,-0.141064,2.456954,-0.046297,-0.105247,-0.027587,-0.042816,-0.148172,-0.048562,-0.103970,-0.182547,-0.006894,-0.062743,-0.171773,-0.084596,-0.189656,-0.039935,-0.030458,0.0,-0.237009,-0.160442,-0.026711,-0.057153,-0.201510,-0.154418,-0.205374,-0.265918,-0.124669,-0.181153,-0.044456,-0.075897,-0.05303,-0.129686,-0.172505,-0.186063,-0.192196,-0.083449,-0.218893,-0.040527,-0.039935,-0.048068,-0.095967,-0.083304,-0.145895,-0.157059,-0.126233,-0.255152,-0.041687,-0.111967,-0.012899,-0.127201,-0.17367,-0.141064,2.456954,-0.046297,-0.027587,-0.105247,-0.042816,-0.048562,-0.148172,-0.103970,-0.018884
29950,0.173118,-0.408188,-1.360673,-0.085786,-0.060007,-0.109951,-0.027623,-1.346416,-0.275594,-1.106666,1.117762,-0.388006,-0.458624,1.393021,-1.430112,-0.012899,0.023386,-0.019503,0.0,-0.006894,-0.182547,-0.171773,-0.062743,-0.084596,-0.189656,-0.039935,0.0,-0.030458,-0.237009,-0.160442,-0.026711,-0.205311,-0.057361,-0.201510,-0.154418,-0.265918,-0.124669,-0.181153,-0.05303,-0.075897,-0.044456,-0.129686,-0.172505,-0.192196,-0.186063,-0.083449,-0.145895,-0.157059,-0.218893,-0.039935,-0.048068,-0.095967,-0.040527,-0.083304,-0.126233,-0.255152,-0.041687,-0.111967,-0.018884,-0.012899,-0.127201,-0.17367,-0.141064,2.456954,-0.046297,-0.105247,-0.027587,-0.042816,-0.148172,-0.048562,-0.103970,-0.182547,-0.006894,-0.062743,-0.171773,-0.084596,-0.189656,-0.039935,-0.030458,0.0,-0.237009,-0.160442,-0.026711,-0.057153,-0.201510,-0.154418,-0.205374,-0.265918,-0.124669,-0.181153,-0.044456,-0.075897,-0.05303,-0.129686,-0.172505,-0.186063,-0.192196,-0.083449,-0.218893,-0.040527,-0.039935,-0.048068,-0.095967,-0.083304,-0.145895,-0.157059,-0.126233,-0.255152,-0.041687,-0.111967,-0.012899,-0.127201,-0.17367,-0.141064,2.456954,-0.046297,-0.027587,-0.105247,-0.042816,-0.048562,-0.148172,-0.103970,-0.018884
24865,-0.164256,-0.826495,1.388857,-0.085786,-0.060007,-0.109951,-0.027623,-0.961690,-0.277887,-1.106666,1.117762,-0.352462,-0.354092,-1.455563,-0.130993,-0.012899,0.023386,-0.019503,0.0,-0.006894,-0.182547,-0.171773,-0.062743,-0.084596,-0.189656,-0.039935,0.0,-0.030458,-0.237009,-0.160442,-0.026711,-0.205311,-0.057361,-0.201510,-0.154418,-0.265918,-0.124669,-0.181153,-0.05303,-0.075897,-0.044456,-0.129686,-0.172505,-0.192196,-0.186063,-0.083449,-0.145895,-0.157059,-0.218893,-0.039935,-0.048068,-0.095967,-0.040527,-0.083304,-0.126233,-0.255152,-0.041687,-0.111967,-0.018884,-0.012899,-0.127201,-0.17367,-0.141064,2.456954,-0.046297,-0.105247,-0.027587,-0.042816,-0.148172,-0.048562,-0.103970,-0.182547,-0.006894,-0.062743,-0.171773,-0.084596,-0.189656,-0.039935,-0.030458,0.0,-0.237009,-0.160442,-0.026711,-0.057153,-0.20

In [18]:
X_train.describe()

,yr,mo,dy,inj,fat,loss,closs,slat,slon,elat,elon,len,wid,hour,minute,tz_0,tz_3,tz_6,tz_9,st_AK,st_AL,st_AR,st_AZ,st_CA,st_CO,st_CT,st_DC,st_DE,st_FL,st_GA,st_HI,st_IA,st_ID,st_IL,st_IN,st_KS,st_KY,st_LA,st_MA,st_MD,st_ME,st_MI,st_MN,st_MO,st_MS,st_MT,st_NC,st_ND,st_NE,st_NH,st_NJ,st_NM,st_NV,st_NY,st_OH,st_OK,st_OR,st_PA,st_PR,st_RI,st_SC,st_SD,st_TN,st_TX,st_UT,st_VA,st_VT,st_WA,st_WI,st_WV,st_WY,stf_1,stf_2,stf_4,stf_5,stf_6,stf_8,stf_9,stf_10,stf_11,stf_12,stf_13,stf_15,stf_16,stf_17,stf_18,stf_19,stf_20,stf_21,stf_22,stf_23,stf_24,stf_25,stf_26,stf_27,stf_28,stf_29,stf_30,stf_31,stf_32,stf_33,stf_34,stf_35,stf_36,stf_37,stf_38,stf_39,stf_40,stf_41,stf_42,stf_44,stf_45,stf_46,stf_47,stf_48,stf_49,stf_50,stf_51,stf_53,stf_54,stf_55,stf_56,stf_72
count,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,42079.0,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,42079.0,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,42079.0,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04,4.207900e+04
mean,9.353113e-16,1.409975e-17,3.470057e-17,9.118398e-18,3.377185e-18,1.350874e-18,-1.182015e-18,8.285922e-16,4.638563e-16,2.549774e-17,1.393089e-16,3.394070e-17,4.559199e-18,6.720597e-17,-8.223444e-17,1.165129e-17,1.972529e-15,3.377185e-19,0.0,3.334970e-18,-6.349107e-17,-7.514236e-18,-1.063813e-17,-4.913803e-17,7.987041e-17,6.754369e-18,0.0,-2.887493e-17,-3.647359e-17,-3.765561e-17,-9.624976e-18,1.418417e-17,2.330257e-17,2.566660e-17,-5.386609e-17,2.718634e-17,-2.988808e-17,1.384646e-17,5.023562e-17,-3.090124e-17,1.004712e-17,2.870607e-18,-2.410465e-17,4.343904e-17,-1.046927e-17,-1.190458e-17,3.934420e-17,-2.532888e-17,-7.041430e-17,4.896918e-18,3.077459e-17,2.397801e-17,8.442961e-19,6.247791e-18,2.532888e-19,5.327509e-17,1.612606e-17,1.494404e-17,-8.780680e-18,1.013155e-17,-1.317102e-17,-1.215786e-17,-4.052621e-17,3.533379e-17,7.683095e-18,4.137051e-18,-1.249558e-17,3.377185e-17,5.319066e-17,-3.546044e-17,2.026311e-18,-6.349107e-17,3.334970e-18,-1.063813e-17,-7.514236e-18,-4.913803e-17,7.987041e-17,6.754369e-18,-2.887493e-17,0.0,-3.647359e-17,-3.765561e-17,-9.624976e-18,-5.183978e-17,2.566660e-17,-5.386609e-17,5.437267e-17,2.718634e-17,-2.988808e-17,1.384646e-17,1.004712e-17,-3.090124e-17,5.023562e-17,2.870607e-18,-2.410465e-17,-1.046927e-17,4.343904e-17,-1.190458e-17,-7.041430e-17,8.442961e-19,4.896918e-18,3.077459e-17,2.397801e-17,6.247791e-18,3.934420e-17,-2.532888e-17,2.532888e-19,5.327509e-17,1.612606e-17,1.494404e-17,1.013155e-17,-1.317102e-17,-1.215786e-17,-4.052621e-17,3.533379e-17,7.683095e-18,-1.249558e-17,4.137051e-18,3.377185e-17,-3.546044e-17,5.319066e-17,2.026311e-18,-8.780680e-18
std,1.000012e+00,1.000012e+0

In [19]:
y_train.value_counts()

mag
0    19482
1    14239
2     6249
3     1682
4      385
5       42
Name: count, dtype: int64

### Training

In [20]:
models = {
    '    Linear Regression': LinearRegression(),
    'Ridge (L2) Regression': Ridge(),
    'Lasso (L1) Regression': Lasso()
}

In [21]:
for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")

    Linear Regression trained.
Ridge (L2) Regression trained.
Lasso (L1) Regression trained.


### Results

In [22]:
for name, model in models.items():
    print(name + " R^2 Score: {:.5f}".format(model.score(X_test, y_test)))

    Linear Regression R^2 Score: 0.40817
Ridge (L2) Regression R^2 Score: 0.40817
Lasso (L1) Regression R^2 Score: -0.00001
